# ES|QL concepts

In [1]:
from elasticsearch import Elasticsearch
import os

# Login details
ES_URL=os.getenv("ES_URL","http://elasticsearch:9200") #""
KB_URL=os.getenv("KB_URL","http://kibana:5601") #"https://foss4geurope.kb.us-central1.gcp.cloud.es.io"

# API key is preferred but some Kibana operations require user/password
ES_APIKEY=os.getenv("ES_APIKEY")
ES_USER=os.getenv("ES_USER","elastic")
ES_PASS=os.getenv("ES_PASS","changeme")
ES_AUTH=(ES_USER, ES_PASS)

# Load the client and some cluster details
client = Elasticsearch(hosts=[ES_URL], api_key=ES_APIKEY)
c_info = client.info()
is_serverless = c_info['version']['build_flavor'] == 'serverless'
print(f"Elasticsearch URL: {ES_URL}")
print(f"Cluster name: {c_info['name']}")
print(f"Version: {c_info['version']['number'] if not is_serverless else "serverless"}")

# Load the ESQL helper
from helpers import esql as esql_query
from functools import partial

esql = partial(esql_query, client=client)

Elasticsearch URL: http://elasticsearch:9200
Cluster name: 85861244b50d
Version: 9.0.3


In [3]:
# Basic query against all places indices, 
# returing the first 10 rows (in no particular order) and
# only retrieving three of the available fields
esql("""
FROM places-*
| KEEP name, category, geometry
| LIMIT 5
""")

,name,category,geometry
0,Rio Guamá,river,POINT (-48.49709 -1.48522)
1,Guaraná do CK,restaurant,POINT (-48.50236 -1.47249)
2,park da orla,active_life,POINT (-48.50226 -1.47229)
3,Sorveteria Ice Point Self-service,ice_cream_shop,POINT (-48.50022 -1.47407)
4,Sushi Expresso Duque,sushi_restaurant,POINT (-48.49968 -1.47358)


In [4]:
# Get also the source index using the METADATA keyword
esql("""
FROM places-* METADATA _index
| KEEP _index, name, category, geometry
| LIMIT 5
""")

,_index,name,category,geometry
0,places-belem,Rio Guamá,river,POINT (-48.49709 -1.48522)
1,places-belem,Guaraná do CK,restaurant,POINT (-48.50236 -1.47249)
2,places-belem,park da orla,active_life,POINT (-48.50226 -1.47229)
3,places-belem,Sorveteria Ice Point Self-service,ice_cream_shop,POINT (-48.50022 -1.47407)
4,places-belem,Sushi Expresso Duque,sushi_restaurant,POINT (-48.49968 -1.47358)


In [5]:
# Rename a field
esql("""
FROM ne_countries
| KEEP name, iso_a2, iso_a3
| RENAME iso_a2 as id2, iso_a3 as id3
| LIMIT 5
""")

,name,id2,id3
0,Indonesia,ID,IDN
1,Malaysia,MY,MYS
2,Chile,CL,CHL
3,Bolivia,BO,BOL
4,Peru,PE,PER


In [6]:
# Basic filter
esql("""
FROM places-* METADATA _index
| KEEP _index, name, category, geometry
| RENAME _index as dataset
| WHERE name LIKE "*Burger*" 
    AND category IN ("restaurant", "burger_restaurant")
| LIMIT 5
""")

,dataset,name,category,geometry
0,places-valencia,The Corner Burger Club,burger_restaurant,POINT (-0.37764 39.45036)
1,places-valencia,Marvin Burgers & Tacos,burger_restaurant,POINT (-0.37672 39.461)
2,places-valencia,Burger Beer Ruzafa,burger_restaurant,POINT (-0.37544 39.46164)
3,places-valencia,The Good Burger,burger_restaurant,POINT (-0.41862 39.47366)
4,places-belem,Caça Burger,burger_restaurant,POINT (-48.47028 -1.44646)


In [7]:
# Sort by a field
esql("""
FROM places-* METADATA _index
| KEEP _index, name, category, geometry
| RENAME _index as dataset
| WHERE name LIKE "*Burger*" 
    AND category IN ("restaurant", "burger_restaurant")
| SORT name
| LIMIT 5
""")

,dataset,name,category,geometry
0,places-belem,AP Burger,burger_restaurant,POINT (-48.49322 -1.46228)
1,places-bosnia,As Burgers,burger_restaurant,POINT (17.90746 44.21046)
2,places-valencia,Asier's Burger Route,burger_restaurant,POINT (-0.3279 39.47242)
3,places-belem,Avalon Burger,restaurant,POINT (-48.47791 -1.46033)
4,places-valencia,Aviador Burger & Beer,burger_restaurant,POINT (-0.34285 39.47354)


In [8]:
# Reverse sort by a field
esql("""
FROM places-* METADATA _index
| KEEP _index, name, category, geometry
| RENAME _index as dataset
| WHERE name LIKE "*Burger*" 
    AND category IN ("restaurant", "burger_restaurant")
| SORT name DESC
| LIMIT 5
""")

,dataset,name,category,geometry
0,places-bosnia,Šah Mat Burger,burger_restaurant,POINT (18.43059 43.85899)
1,places-belem,Vikings Burger - Shopping Pátio Belém,burger_restaurant,POINT (-48.49558 -1.4577)
2,places-belem,Vikings Burger,burger_restaurant,POINT (-48.48357 -1.44605)
3,places-belem,Vikings Burger,burger_restaurant,POINT (-48.45998 -1.42733)
4,places-belem,Veg Burger,burger_restaurant,POINT (-48.48278 -1.44495)


In [9]:
# Aggregate: count
esql("""
FROM ne_countries
| STATS counts = count(id)
| LIMIT 5
""")

,counts
0,246


In [10]:
# Aggregate: count by grouping field
esql("""
FROM ne_countries
| WHERE type in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent
| SORT continent
| KEEP continent, counts
| LIMIT 10
""")

,continent,counts
0,Africa,51
1,Asia,48
2,Europe,48
3,North America,26
4,Oceania,14
5,Seven seas (open ocean),3
6,South America,12


In [11]:
# Aggregate: count by more than one grouping field
esql("""
FROM ne_countries
| WHERE type in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent, type
| SORT continent, type
| KEEP continent, type, counts
| LIMIT 50
""")

,continent,type,counts
0,Africa,Sovereign country,51
1,Asia,Country,3
2,Asia,Sovereign country,45
3,Europe,Country,9
4,Europe,Sovereign country,39
5,North America,Country,5
6,North America,Sovereign country,21
7,Oceania,Country,2
8,Oceania,Sovereign country,12
9,Seven seas (open ocean),Sovereign country,3


In [12]:
# Compute new fields
esql("""
FROM ne_countries
| WHERE gdp_md IS NOT NULL 
    AND pop_est > 0
    AND type IN ("Country", "Sovereign country")
| EVAL gdp_pop = ROUND(( gdp_md * 1e6) / ( pop_est::double))::integer
| SORT gdp_pop DESC
| KEEP name, type, gdp_md, pop_est, gdp_pop
| LIMIT 10
""")

,name,type,gdp_md,pop_est,gdp_pop
0,Monaco,Sovereign country,7188,38964,184478
1,Liechtenstein,Sovereign country,6876,38019,180857
2,Luxembourg,Sovereign country,71104,619896,114703
3,Isle of Man,Country,7491,84584,88563
4,Macao,Country,53859,640445,84096
5,Switzerland,Sovereign country,703082,8574832,81994
6,Ireland,Sovereign country,388698,4941444,78661
7,Norway,Sovereign country,403336,5347896,75420
8,Iceland,Sovereign country,24188,361313,66945
9,United States of America,Country,21433226,328239523,65298
